In [11]:
import subprocess
from pydub import AudioSegment


def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-i",
        video_path,
        "-vn",
        audio_path,
    ]
    subprocess.run(command)


track = AudioSegment.from_mp3("./files/podcast.mp3")

ten_minutes = 10 * 60 * 1000

In [18]:
import math

chunks = math.ceil(len(track) / ten_minutes)

for i in range(chunks):
    start_time = i * ten_minutes
    end_time = (i + 1) * ten_minutes

    chunk = track[start_time:end_time]

    chunk.export(f"./files/chunks/chunk_{i}.mp3", format="mp3")